# **Intercomparison Demo**

**Author:** Adam Yang (ayang115@umd.edu) and Hannah Kerner (hkerner@asu.edu)

**Description** Demo for Intercomparison Script: extracts manually labeled test points from publicly-available land cover maps (Harvest, Copernicus, ESA, GLAD, etc) and evaluates performance metrics for each map.

In [ ]:
# !earthengine authenticate

In [1]:
import ee
ee.Initialize()

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ee
from shapely import wkt

import sys
sys.path.append("..")
from src.compare_covermaps import *

In [3]:
maps = [v for v in TARGETS.values()]
countries = ["Togo", "Kenya", "Malawi", "Tanzania", "Mali", "Namibia", "Rwanda", "Uganda", "Zambia"]

In [4]:
covermap_test = TestCovermaps(test_countries=countries, covermaps=maps)

In [5]:
# Load the test data points
test_pts = covermap_test.get_test_points()

# Alternatively, read from existing file
# test_pts = pd.read_csv("../../intercomparison-data/extracted/test/tgo_ken_tza_mlw_test.csv").drop("Unnamed: 0", axis=1)
# test_pts["geometry"] = test_pts["geometry"].apply(wkt.loads)
# test_pts = gpd.GeoDataFrame(test_pts, crs="epsg:4326")

# Print a random sample of the dataframe
test_pts.sample(10)

/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/Users/hkerner/anaconda3/envs/landcover-mapping/l

,lat,lon,binary,country,subset,geometry
3222,2.198303,34.919666,0,Uganda,testing,POINT (34.91967 2.19830)
293,1.133771,38.170881,0,Kenya,validation,POINT (38.17088 1.13377)
3792,-17.623720,26.849855,0,Zambia,testing,POINT (26.84986 -17.62372)
359,-0.650380,35.198015,1,Kenya,validation,POINT (35.19802 -0.65038)
162,10.561161,0.313722,1,Togo,testing,POINT (0.31372 10.56116)
1168,-7.189240,37.468436,0,Tanzania,testing,POINT (37.46844 -7.18924)
687,-1.379029,37.942212,0,Kenya,testing,POINT (37.94221 -1.37903)
2696,-17.620797,17.664289,0,Namibia,testing,POINT (17.66429 -17.62080)
3325,1.333915,32.085961,0,Uganda,testing,POINT (32.08596 1.33392)
1797,-2.992968,31.029933,0,Tanzania,testing,POINT (31.02993 -2.99297)


In [6]:
extracted = covermap_test.extract_covermaps(test_pts)

[Togo] sampling copernicus...
[Togo] sampling worldcover...
[Togo] sampling glad...
[Togo] sampling asap...
[Togo] sampling dynamicworld...
[Togo] sampling gfsad-gcep...
Extracting error: length of sampled dataset isnot the same as testing dataset (more)
[Togo] sampling gfsad-lgrip...
[Togo] sampling digital-earth-africa...
[Togo] sampling esa-cci-africa...
[Togo] sampling globcover-v23...
[Togo] sampling globcover-v22...
[Togo] sampling harvest-crop-maps...
[Togo] sampling esri-lulc...
Extracting error: length of sampled dataset isnot the same as testing dataset (more)
[Kenya] sampling copernicus...
[Kenya] sampling worldcover...
[Kenya] sampling glad...
[Kenya] sampling asap...
[Kenya] sampling dynamicworld...
[Kenya] sampling gfsad-gcep...
Extracting error: length of sampled dataset isnot the same as testing dataset (more)
[Kenya] sampling gfsad-lgrip...
Extracting error: length of sampled dataset isnot the same as testing dataset (more)
[Kenya] sampling digital-earth-africa...
[Ken

In [7]:
_ = covermap_test.evaluate()
results = pd.concat(covermap_test.results)
results

evaluating maps...
dataset: copernicus | country: Togo
dataset: worldcover | country: Togo
dataset: glad | country: Togo
dataset: asap | country: Togo
dataset: dynamicworld | country: Togo
dataset: gfsad-gcep | country: Togo
dataset: gfsad-lgrip | country: Togo
dataset: digital-earth-africa | country: Togo
dataset: esa-cci-africa | country: Togo
dataset: globcover-v23 | country: Togo
dataset: globcover-v22 | country: Togo
dataset: harvest-crop-maps | country: Togo
dataset: esri-lulc | country: Togo
dataset: copernicus | country: Kenya
dataset: worldcover | country: Kenya
dataset: glad | country: Kenya
dataset: asap | country: Kenya
dataset: dynamicworld | country: Kenya
dataset: gfsad-gcep | country: Kenya
dataset: gfsad-lgrip | country: Kenya
dataset: digital-earth-africa | country: Kenya
dataset: esa-cci-africa | country: Kenya
dataset: globcover-v23 | country: Kenya
dataset: globcover-v22 | country: Kenya
dataset: harvest-crop-maps | country: Kenya
dataset: esri-lulc | country: Keny

/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

dataset: digital-earth-africa | country: Namibia
dataset: esa-cci-africa | country: Namibia
dataset: globcover-v23 | country: Namibia
dataset: globcover-v22 | country: Namibia
dataset: esri-lulc | country: Namibia
dataset: copernicus | country: Rwanda
dataset: worldcover | country: Rwanda
dataset: glad | country: Rwanda
dataset: asap | country: Rwanda
dataset: dynamicworld | country: Rwanda
dataset: gfsad-gcep | country: Rwanda
dataset: gfsad-lgrip | country: Rwanda
dataset: digital-earth-africa | country: Rwanda
dataset: esa-cci-africa | country: Rwanda
dataset: globcover-v23 | country: Rwanda
dataset: globcover-v22 | country: Rwanda
dataset: esri-lulc | country: Rwanda
dataset: copernicus | country: Uganda
dataset: worldcover | country: Uganda
dataset: glad | country: Uganda
dataset: asap | country: Uganda
dataset: dynamicworld | country: Uganda
dataset: gfsad-gcep | country: Uganda
dataset: gfsad-lgrip | country: Uganda
dataset: digital-earth-africa | country: Uganda
dataset: esa-cc

country  crop_f1  accuracy  crop_support  \
       dataset                                                         
Togo   copernicus              Togo     0.59      0.79            51   
       worldcover              Togo     0.70      0.86            51   
       glad                    Togo     0.71      0.86            51   
       asap                    Togo     0.45      0.64            51   
       dynamicworld            Togo     0.21      0.75            51   
...                             ...      ...       ...           ...   
Zambia digital-earth-africa  Zambia     0.57      0.94            20   
       esa-cci-africa        Zambia     0.46      0.93            20   
       globcover-v23         Zambia     0.19      0.91            20   
       globcover-v22         Zambia     0.10      0.85            20   
       esri-lulc             Zambia     0.74      0.98            20   

                             noncrop_support  crop_recall  noncrop_recall  \
       dataset                                                              
Togo   copernicus                        131         0.55            0.88   
       worldcover                        131         0.57            0.98   
       glad                              131         0.59            0.97   
       asap                              131         0.53            0.68   
       dynamicworld                      131         0.12            0.99   
...                                      ...          ...             ...   
Zambia digital-earth-africa              469         0.95            0.94   
       esa-cci-africa                    469         0.70            0.94   
       globcover-v23                     469         0.25            0.94   
       globcover-v22                     469         0.20            0.88   
       esri-lulc                         469         0.70            0.99   

                             crop_precision  noncrop_precision  
       dataset                                                  
Togo   copernicus                      0.64               0.83  
       worldcover                      0.91               0.85  
       glad                            0.88               0.86  
       asap                            0.39               0.79  
       dynamicworld                    0.86               0.74  
...                                     ...                ...  
Zambia digital-earth-africa            0.40               1.00  
       esa-cci-africa                  0.34               0.99  
       globcover-v23                   0.15               0.97  
       globcover-v22                   0.07               0.96  
       esri-lulc                       0.78               0.99  

[111 rows x 9 columns]

In [8]:
results.to_csv("../notebooks/intercomparison-results.csv")